### Индивидуальное задание.

#### Через все точки пересечения графиков функций f(x) и g(x) провести горизонтальные прямые. Изобразить на графике f(x) зеленым цветом, g(x) красным, горизонтальные линии черным. Формулы для f(x) и g(x) включить в легенду (например, как f(x) = 3x^2 + 5x − 7).

In [11]:
import numpy as np
import sympy as sp
from sympy.abc import x
import matplotlib.pyplot as plt
import os
%matplotlib inline                     

In [12]:
%run document_generator.ipynb

In [31]:
"""
Returns two "nearby placed" functions and two points, at which each function intersect other.
"""
def generate_functions() -> tuple[sp.Function, sp.Function, sp.Point2D, sp.Point2D]:
    def coef(hi, lo = 1):
        return sp.sympify(np.random.choice([-1, 1]) * np.random.randint(lo, hi + 1))

    while True:
        a = coef(3) / coef(2)
        b = coef(6, 0) / coef(2)
        f = a*x**2 + b*x + coef(10, 0) / coef(2)
        
        x_mid = -b / (2 * a)
        x1 = x_mid - np.random.randint(1, 3) 
        x2 = x_mid + np.random.randint(1, 3)

        y1 = f.subs({x: x1})
        y2 = f.subs({x: x2})

        # Disallow similar heights
        if y1 - y2 < 1:
            continue

        solutions = sp.linsolve((sp.Matrix([[x1**2, x1, 1], [x2**2, x2, 1]]), sp.Matrix([y1, y2]))).args
        assert len(solutions) > 0, "No solutions found"

        a, b, c = solutions[0]
        free = c if isinstance(c, sp.Symbol) else (b if b != isinstance(b, sp.Symbol) else a)

        substitute = np.random.randint(-10, 11)
        a = a.subs({free: substitute})
        if a == 0:
            continue 

        b = b.subs({free: substitute})
        c = c.subs({free: substitute})

        g = a*x**2 + b*x + c
        if f == g:
            continue

        f, g = np.random.permutation([f, g])

        if np.random.randint(2) == 0:
            f = sp.factor(f)

        if np.random.randint(2) == 0:
            g = sp.factor(g)

        return f, g, sp.Point2D(x1, y1), sp.Point(x2, y2)

In [14]:
PICS_PATH = "../out/answers/answer3/"

In [29]:
def create_plot(f: sp.Function, 
                g: sp.Function,
                p1: sp.Point2D,
                p2: sp.Point2D, 
                name: str):
    def plot(f, line_color: str, label: str = ""):
        return sp.plot(f,
                       xlim=x_limits,
                       ylim=y_limits, 
                       show=False, 
                       legend=True, 
                       line_color=line_color,
                       grid=True,
                       label=label)

    x_limits = (sp.Min(p1[0], p2[0]) - 3, sp.Max(p1[0], p2[0]) + 3)
    y_limits = (sp.Min(p1[1], p2[1]) - 3, sp.Max(p1[1], p2[1]) + 3)

    p = plot(f, "green", f"$f(x) = {sp.latex(f)}$")
    p.extend(plot(g, "red", f"$g(x) = {sp.latex(g)}$"))
    p.extend(plot(p1[1], "black"))
    p.extend(plot(p2[1], "black"))

    os.makedirs(os.path.dirname(PICS_PATH), exist_ok=True)
    p.save(PICS_PATH + name)
    
    # Close backend matplotlib plot to save on resources.
    plt.close("all")

In [16]:
HEADER = f"""\\documentclass[11pt]{{report}}
\\usepackage[T2A]{{fontenc}}
\\usepackage[utf8]{{inputenc}}
\\usepackage[russian]{{babel}}
\\usepackage{{amsmath,amssymb}}
\\usepackage{{graphicx}}
\\oddsidemargin=-19mm
\\topmargin=-30mm
\\textheight 26cm
\\hsize 18cm
\\textwidth 20cm

\\begin{{document}}

\\pagestyle{{empty}}"""

DESCRIPTION = """Через все точки пересечения графиков функций $f(x)$ и $g(x)$ провести горизонтальные прямые. 
Изобразить на графике $f(x)$ зеленым цветом, $g(x)$ красным, горизонтальные линии черным. 
Формулы для $f(x)$ и $g(x)$ включить в легенду (например, как $f(x) = 3x^2 + 5x - 7$)."""

TASK = """\\begin{{align*}}
    f(x) = {f} && g(x) = {g}
\\end{{align*}}"""

SOLUTION = """\\begin{{align*}}
    f(x) = {f} && g(x) = {g}
\\end{{align*}}
Point 1: ${p1}$
Point 2: ${p2}$

\\includegraphics[scale=0.8]{{ {{ {plot_name} }} }}
"""

In [17]:
def variant_generator(variant_n: int):
    f, g, p1, p2 = generate_functions() 

    name = f"{variant_n}.png"
    create_plot(f, g, p1, p2, name)

    f_latex = sp.latex(f)
    g_latex = sp.latex(g)

    return TASK.format(f=f_latex, g=g_latex), SOLUTION.format(f=f_latex, g=g_latex, p1=sp.latex(p1), p2=sp.latex(p2), plot_name=name)

DOC = DocumentGenerator(variant_generator, DESCRIPTION, header = HEADER)

In [32]:
write_tasks_and_solutions(DOC, "../out/tasks/task-3.tex", "../out/answers/answer3/answer-3.tex", 150)